# Cvičení 4a

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat LDLT rozkladem.

# LDLT rozklad

LDLT rozklad je vhodný pro symetrické matice. Lze jej popsat následujícím pseudokódem:

```
n = size(A)
L = I, D = 0

v(1) = A(1, 1)
D(1, 1) = v(1)
L(2:n, 1) = A(2:n, 1)/v(1)

for j = 2 to n do
    for i = 1 to j - 1 do
        v(i) = L(j, i)D(i, i)
    end

    v(j) = A(j, j) - L(j, 1:j-1)v(1:j-1)
    D(j, j) = v(j)
    L(j+1:n, j) = (A(j+1:n, j) - L(j+1:n, 1:j-1)v(1:j-1)) / v(j)
end
```

V následující části implementujete LDLT rozklad a využijete ho k řešení soustavy se symetrickou maticí.

In [ ]:
# ÚKOL: Doplňte následující kód

import numpy as np

def my_ldlt(A):
    """
    Vypočítá LDLT rozklad symetrické matice.
    Vstup: A - A symetrická matice
    Výstup: L, D - Matice dekompozice takové, že platí A = L*D*L.T
    """
    m, n = A.shape

    # Zkontrolujme, že je matice čtvercová
    if m != n:
        raise ValueError('The matrix must be square!')

    L = np.eye(m)
    D = np.zeros((m, m))

    v = np.zeros(m)
    v[0] = A[0, 0]
    D[0, 0] = v[0]
    L[1:n, 0] = A[1:n, 0] / v[0]

    for j in range(1, n):
        for i in range(j):
            v[i] = L[j, i] * D[i, i]

        v[j] = A[j, j] - np.dot(L[j, 0:j], v[0:j])
        D[j, j] = v[j]
        if j+1 <= n-1:  # Abychom v poslední iteraci nepřesáhli meze
            L[j+1:n, j] = (A[j+1:n, j] - np.dot(L[j+1:n, 0:j], v[0:j])) / v[j]

    return L, D

In [ ]:
# Vygenerujeme náhodnou symetrickou matici
A = np.random.rand(5, 5)
A = A + A.transpose()
print(A)

In [ ]:
# Otestujeme, že váš kód vrací správný výsledek
L, D = my_ldlt(A)
AA = L@D@L.transpose()
print(AA-A)

Známe-li rozklad symetrické matice $\mathsf{A} = \mathsf{L}\mathsf{D}\mathsf{L}^T$, můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$, kterou převedeme na 
$$\mathsf{L}\mathsf{D}\mathsf{L}^T\mathbf{x} = \mathbf{b}$$
a vyřešíme vhodnými substitucemi s využitím dopředné a zpětné substituce.

Rozmyslete si, jak takové substituce budou vypadat a v následující části vyřešte náhodnou soustavu.

In [ ]:
# ÚKOL: Zkopírujte z minulého cvičení

def fsubst(L, b):
    m, n = np.shape(L)
    x = np.zeros(n)

    x[0] = b[0] / L[0, 0]

    for i in range(1, m):
        suma = 0
        for j in range(0, i):
            suma += L[i, j] * x[j]

        x[i] = ( b[i] - suma ) / L[i, i]

    return x

In [ ]:
# ÚKOL: Zkopírujte z minulého cvičení

def bsubst(U, b):

    m, n = np.shape(U)
    x = np.zeros(n)

    x[n-1] = b[n-1]/U[n-1, n-1]

    for i in range(n-2, -1, -1):
        suma = 0
        for j in range(i+1, n):
            suma += U[i, j] * x[j]

        x[i] = (b[i] - suma) / U[i, i]

    return x

In [ ]:
# ÚKOL: Doplňte následující buňku a vyřešte soustavu rovnic Ax = b se symetrickou náhodnou maticí A a
# náhodným vektorem pravé strany b. Využijte vaše metody fsubst a bsubst
b = np.random.rand(5)
y = fsubst(L, b)

z = np.zeros(b.size)
for i in range(b.size):
    z[i] = y[i] / D[i, i]

vase_x = bsubst(L.transpose(), z)

In [ ]:
# Porovnáme vaše řešení a řešen pomocí numpy
print(vase_x)
print(np.linalg.solve(A, b))